# MLOps Coloring Book

This notebook may be used for demonstration of AutoMLOps

## Imports

In [1]:
# AutoMLOps
from utils import AutoMLOps

In [2]:
%%define_imports
import json
import pandas as pd
from google.cloud import aiplatform
from google.cloud import aiplatform_v1
from google.cloud import bigquery
from google.cloud import storage
import datetime
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split
from joblib import dump
import pickle
import os

## Globals

In [3]:
bq_table = "sandbox-srastatter.mlops_boxer_test.dry-beans"
model_directory = f"gs://mlops-boxer-test/trained_models/{datetime.datetime.now()}"
data_path = "gs://mlops-boxer-test/data"
project_id = "sandbox-srastatter"
region = "us-central1"

## Data Loading

In [4]:
%%define_component
AutoMLOps.makeComponent(
    name="create_dataset",
    description="Loads data from BQ and writes a dataframe as a csv to GCS.", # optional
    params=[
        {"name": "bq_table", "type": str, "description": "The bq input table."}, # descriptions are optional
        {"name": "data_path", "type": str, "description": "GS location where the training data is written."},
        {"name": "project_id", "type": str, "description": "Project_id."}
    ]
)

bq_client = bigquery.Client(project=project_id)

def get_query(bq_input_table: str) -> str:
    """Generates BQ Query to read data.

    Args:
    bq_input_table: The full name of the bq input table to be read into
    the dataframe (e.g. <project>.<dataset>.<table>)
    Returns: A BQ query string.
    """
    return f"""
    SELECT *
    FROM `{bq_input_table}`
    """

def load_bq_data(query: str, client: bigquery.Client) -> pd.DataFrame:
    """Loads data from bq into a Pandas Dataframe for EDA.
    Args:
    query: BQ Query to generate data.
    client: BQ Client used to execute query.
    Returns:
    pd.DataFrame: A dataframe with the requested data.
    """
    df = client.query(query).to_dataframe()
    return df

dataframe = load_bq_data(get_query(bq_table), bq_client)
dataframe.to_csv(data_path)

## Model Training

In [8]:
%%define_component
AutoMLOps.makeComponent(
    name="train_model",
    description="Trains a decision tree on the training data.",
    params=[
        {"name": "model_directory", "type": str, "description": "GS location of saved model."},
        {"name": "data_path", "type": str, "description": "GS location where the training data."}
    ]
)

def save_model(model, model_directory):
    """Saves a model to uri."""
    filename = f'model.pkl'
    with open(filename, 'wb') as f:
        pickle.dump(model, f)
    
    bucket_name = model_directory.split('/')[2]
    prefix='/'.join(model_directory.split('/')[3:])
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(os.path.join(prefix, filename))
    blob.upload_from_filename(filename)

df = pd.read_csv(data_path)
labels = df.pop("Class").tolist()
data = df.values.tolist()
x_train, x_test, y_train, y_test = train_test_split(data, labels)
skmodel = DecisionTreeClassifier()
skmodel.fit(x_train,y_train)
score = skmodel.score(x_test,y_test)
print('accuracy is:',score)

output_uri = os.path.join(model_directory, f'model.pkl')
save_model(skmodel, model_directory)

## Uploading & Deploying the Model

In [9]:
%%define_component
AutoMLOps.makeComponent(
    name="deploy_model",
    description="Trains a decision tree on the training data.",
    params=[
        {"name": "model_directory", "type": str, "description": "GS location of saved model."},
        {"name": "project_id", "type": str, "description": "Project_id."},
        {"name": "region", "type": str, "description": "Region."}
    ]
)

aiplatform.init(project=project_id, location=region)
deployed_model = aiplatform.Model.upload(
    display_name="beans-model-pipeline",
    artifact_uri = model_directory,
    serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-24:latest"
)
endpoint = deployed_model.deploy(machine_type="n1-standard-4")

## Define and Run the Pipeline

In [10]:
pipeline_params = {
    "bq_table": bq_table,
    "model_directory": model_directory,
    "data_path": data_path,
    "project_id": project_id,
    "region": region
}

In [11]:
AutoMLOps.makePipeline(
    name="training-pipeline",
    description="description", # optional
    params=[
        {"name": "bq_table", "type": str, "description": "Description."}, # descriptions are optional
        {"name": "model_directory", "type": str, "description": "Description."},
        {"name": "data_path", "type": str, "description": "Description."},
        {"name": "project_id", "type": str, "description": "Description."},
        {"name": "region", "type": str, "description": "Description."}
    ],
    pipeline=[{
        "component_name": "create_dataset", "param_mapping": [
            ("bq_table", "bq_table"), # (component_param, pipeline_param)
            ("data_path", "data_path"),
            ("project_id", "project_id")
        ]
    },
    {
        "component_name": "train_model", "param_mapping": [
            ("model_directory", "model_directory"),
            ("data_path", "data_path")
        ]
    },
    {
        "component_name": "deploy_model", "param_mapping": [
            ("model_directory", "model_directory"),
            ("project_id", "project_id"),
            ("region", "region")
        ]
    }]
)

In [12]:
AutoMLOps.go(project_id='sandbox-srastatter', pipeline_params=pipeline_params)